In [ ]:
from pathlib import Path

from result import Ok, Err, Result

from benthoscan.backends import metashape as backend
from benthoscan.utils.log import logger

In [ ]:
PATHS: dict[str, Path] = {
    "DOCUMENT_IN": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
    ),
    "DOCUMENT_OUT": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects_test/r23685bc_working_version_saved.psz"
    ),
    "CACHE": Path("/home/martin/dev/benthoscan/.cache/"),
}

result: Result[str, str] = backend.load_project(PATHS.get("DOCUMENT_IN"))
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

backend.log_internal_data()

In [ ]:
from pathlib import Path

import Metashape
import numpy as np
import plotly.express as px

from tqdm import tqdm

import benthoscan.io as io

from benthoscan.backends.metashape import (
    render_range_and_normal_maps,
    compute_camera_calibration,
)

def export_rendered_range(chunk: Metashape.Chunk, range_directory: Path, normal_directory: Path) -> None:
    """Export range maps rendered from a Metashape mesh."""
    
    valid_cameras: list[Metashape.Camera] = [camera for camera in chunk.cameras if camera.transform]

    logger.info(f"Range directory: {range_directory}")
    logger.info(f"Normal directory: {normal_directory}")
    
    for camera in tqdm(valid_cameras, desc="Exporting rendered range maps..."):
        
        geometry: tuple[Image, Image] = render_range_and_normal_maps(camera)
        range_map, normal_map = geometry
        
        range_map.data: np.ndarray = range_map.data.astype(np.float16)
        normal_map.data: np.ndarray = normal_map.data.astype(np.float16)
        
        io.write_image(range_directory / f"{camera.label}.tiff", range_map.to_array())
        io.write_image(normal_directory / f"{camera.label}.tiff", normal_map.to_array())

### Export range maps from Metashape

In [ ]:
from typing import NamedTuple

document: Metashape.Document = backend.context._backend_data.get("document")

target_labels: list[str] = [ "r23685bc_20100605_021022" ]
target_chunks: list[Metashape.Chunk] = [chunk for chunk in document.chunks if chunk.label in target_labels]

output_root: Path = Path("/data/kingston_snv_01")

# Export rendered depth
for chunk in target_chunks:
    
    directories: dict[str, Path] = {
        "range_maps": output_root / Path("rendered_range_maps") / f"{chunk.label}",
        "normal_maps": output_root / Path("rendered_normal_maps") / f"{chunk.label}",
    }
    
    export_rendered_range(
        chunk, 
        range_directory=directories.get("range_maps"),
        normal_directory=directories.get("normal_maps"),
    )